# Challenge Assignment
## Cliff Walking with Reinforcement Learning

## CSCI E-82A

>**Make sure** you include your name along with the name of your team and team members in the notebook you submit.

## Introduction

In this challenge you will apply several reinforcement learning algorithms to a classic problem in reinforcement learning, known as the cliff walking problem. The cliff walking problem is basically a game. The goal is for the agent to find the highest reward (lowest cost) path from a starting state to the goal. 

There are a number of versions of the cliff walking problems which have been used as research benchmarks over the years. A typical cliff walking problem might use a grid of 4x12. For this challenge you will work with a reduced size grid world of 4x4 illustrated below to reduce training time for your models.   

<img src="CliffWalking.JPG" alt="Drawing" style="width:200px; height:200px"/>
<center> **Grid World for similified cliff walking problem** </center>

The goal is to find the highest reward path from the **starting state**, 12, to the **terminal state**, 15, making this an **episodic task**. The rewards for this task are:
1. A reward of -1 for most state transitions. The -1 reward apples to state to state transitions and to transitions toward the boundary of the grid transitioning to the same state.    
2. A reward of -100 for 'falling off the cliff'. Falling off the cliff occurs when entering states 13 or 14. The only possible transition out of the cliff states is back to the origin state, 12. There are no possible transitions toward the boundary from the cliff state. 

Intuitively, we can see that the optimal solution follows the dotted line path shown in the diagram above. The challenge is to find a path that is as close to this optimal as possible.   

You can find a short discussion of the cliff walking problem on page 132 of Sutton and Barto, second edition. 

## Challenge

For this challenge you will do the following:

1. Create a simulator for the grid world environment. All interactions between your agents and the environment must be though calls to the function you create.  
2. Create and apply state value estimation agents using the RL algorithms.
3. Use the general policy improvement (GPI) algorithm with the appropriate control algorithm to improve policy. 
4. Use the state value estimation agent to evaluate the improved policy. 
5. Compare the results for the various control algorithms you try. 

Methods to use to solve this problem:

1. The Monte Carlo method for value estimation and (action value) control. The action value method for Monte Carlo has not been explicitly addressed in this course. You can find the pseudo code for Monte Carlo control on page 101 of Sutton and Barto, second edition.   
2. Create and execute agents using the n-step TD method for value estimation and n-step SARSA (action value) for control.
3. Create and execute agents using TD(0) for value estimation and SARSA(0) or Double Q-Learning (action value) control. You are welcome to try both algorithms if you have the time. 
4. For additional, but optional, challenge you may wish to try a dynamic programming algorithm. Does DP work for this problem or not, and why? 

> **Hints**
> - For TD(0), n-step TD, n-step SARSA, SARSA(0) and Double Q-learning, you may need to change the reward to -10 for state transitions toward the boundary of the grid world.  
> - For the n-step algorithms keep in mind that the grid world is rather small. 
> - Make sure you are not accidentally using two epsilon greedy steps in your GPI process.  

## 1.The Monte Carlo method for value estimation and (action value) control. The action value method for Monte Carlo has not been explicitly addressed in this course. You can find the pseudo code for Monte Carlo control on page 101 of Sutton and Barto, second edition.

In [1]:
import numpy as np
import numpy.random as nr

## Define the transition dictonary of dictionaries:
neighbors={0:{'u':0, 'd':4, 'l':0, 'r':1},
          1:{'u':1, 'd':5, 'l':0, 'r':2},
          2:{'u':2, 'd':6, 'l':1, 'r':3},
          3:{'u':3, 'd':7, 'l':2, 'r':3},
          4:{'u':0, 'd':8, 'l':4, 'r':5},
          5:{'u':1, 'd':9, 'l':4, 'r':6},
          6:{'u':2, 'd':10, 'l':5, 'r':7},
          7:{'u':3, 'd':11, 'l':6, 'r':7},
          8:{'u':4, 'd':12, 'l':8, 'r':9},
          9:{'u':5, 'd':13, 'l':8, 'r':10},
          10:{'u':6, 'd':14, 'l':9, 'r':11},
          11:{'u':7, 'd':15, 'l':10, 'r':11},
          12:{'u':8, 'd':12, 'l':12, 'r':13},
          13:{'u':12, 'd':12, 'l':12, 'r':12},
          14:{'u':12, 'd':12, 'l':12, 'r':12},
          15:{'u':15, 'd':15, 'l':15, 'r':15}}

In [2]:
policy =               {0:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        1:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25}, 
                        2:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        3:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        4:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        5:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        6:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        7:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        8:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        9:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        10:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        11:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        12:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        13:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        14:{'u':0.25, 'd':0.25, 'l':0.25, 'r':0.25},
                        15:{'u':0.00, 'd':0.00, 'l':0.00, 'r':0.00}}


In [3]:
rewards ={0:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          1:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          2:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          3:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          4:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          5:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          6:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          7:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          8:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          9:{'u':-1.0, 'd':-100.0, 'l':-1.0, 'r':-1.0},
          10:{'u':-1.0, 'd':-100.0, 'l':-1.0, 'r':-1.0},
          11:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          12:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-100.0},
          13:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          14:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          15:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0}}

In [407]:
#Rewards with a -10.0 for hitting the wall
rew_wall={0:{'u':-10.0, 'd':-1.0, 'l':-10.0, 'r':-1.0},
          1:{'u':-10.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          2:{'u':-10.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          3:{'u':-10.0, 'd':-1.0, 'l':-1.0, 'r':-10.0},
          4:{'u':-1.0, 'd':-1.0, 'l':-10.0, 'r':-1.0},
          5:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          6:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          7:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-10.0},
          8:{'u':-1.0, 'd':-1.0, 'l':-10.0, 'r':-1.0},
          9:{'u':-1.0, 'd':-100.0, 'l':-1.0, 'r':-1.0},
          10:{'u':-1.0, 'd':-100.0, 'l':-1.0, 'r':-1.0},
          11:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-10.0},
          12:{'u':-1.0, 'd':-10.0, 'l':-10.0, 'r':-100.0},
          13:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          14:{'u':-1.0, 'd':-1.0, 'l':-1.0, 'r':-1.0},
          15:{'u':0.0, 'd':0.0, 'l':0.0, 'r':0.0}}

In [5]:
start_state = 12

In [6]:
def MC_generate_episode(start_state, policy, neighbors, terminal):
    ## List of states which might be visited in episode
    n_states = len(policy)
#    visited_state = [0] * n_states
    states = list(neighbors.keys())
    current_state = start_state
    #while(current_state == terminal): # Keep trying to not use terminal state to start
    #    current_state = nr.choice(states, size = 1)[0]
            
    ## Take a random walk trough the states until we get to the terminal state
    ## We do some bookkeeping to ensure we only visit states once.
    visited = [] # List of states visited on random walk
    while(current_state != terminal): # Stop when at terminal state
        ## Probability of state transition given policy
        probs = list(policy[current_state].values())
        ## Find next state to transition to
        next_state = nr.choice(list(neighbors[current_state].values()), size = 1, p = probs)[0]
        visited.append(next_state)
        current_state = next_state  
    return(visited)    
    
    
#MC_generate_episode(start_state, policy, neighbors, 15)

In [7]:
def MC_state_values(start_state, policy, neighbors, rewards, terminal, episodes = 1):
    '''Function for first visit Monte Carlo on GridWorld.'''
    ## Create list of states 
    states = list(policy.keys())
    n_states = len(states)
    
    ## An array to hold the accumulated returns as we visit states
    G = np.zeros((episodes,n_states))
    
    ## An array to keep track of how many times we visit each state so we can 
    ## compute the mean
    n_visits = np.zeros((n_states))
    
    ## Iterate over the episodes
    for i in range(episodes):
        ## For each episode we use a list to keep track of states we have visited.
        ## Once we visit a state we need to accumulate values to get the returns
        states_visited = []
   
        ## Get a path for this episode
        visit_list = MC_generate_episode(start_state, policy, neighbors, terminal)
        current_state = visit_list[0]
        for state in visit_list[0:]: 
            ## list of states we can transition to from current state
            transition_list = list(neighbors[current_state].values())
            
            if(state in transition_list): # Make sure the transistion is allowed
                transition_index = transition_list.index(state)   
  
                ## find the action value for the state transition
                v_s = list(rewards[current_state].values())[transition_index]
   
                ## Mark that the current state has been visited 
                if(state not in states_visited): states_visited.append(current_state)  
                ## Loop over the states already visited to add the value to the return
                for visited in states_visited:
                    G[i,visited] = G[i,visited] + v_s
                    n_visits[visited] = n_visits[visited] + 1.0
            ## Update the current state for next transition
            current_state = state   
    
    ## Compute the average of G over the episodes are return
    n_visits = [nv if nv != 0.0 else 1.0 for nv in n_visits]
    returns = np.divide(np.sum(G, axis = 0), n_visits)   
    return(returns)              
    
returns = MC_state_values(start_state, policy, neighbors, rewards, terminal = 15, episodes = 500)
np.array(returns).reshape((4,4))

array([[-8.0126417 , -7.87805027, -7.81337851, -7.67918882],
       [-8.04466009, -8.14590834, -7.98329222, -8.26967488],
       [-8.28342024, -8.9028726 , -9.06062143, -8.40720391],
       [-9.33705457, -8.66709983, -8.37936446,  0.        ]])

In [8]:
import copy
def MC_optimal_policy(start_state, policy, neighbors, rewards, terminal, episodes = 10, cycles = 10, epsilon = 0.05):
    ## Create a working cooy of the initial policy
    current_policy = copy.deepcopy(policy)
    
    ## Loop over a number of cycles of GPI
    for _ in range(cycles):
        ## First compute the average returns for each of the states. 
        ## This is the policy evaluation phase
        returns = MC_state_values(start_state, current_policy, neighbors, rewards, terminal = terminal,\
                                  episodes = episodes)
        
        ## We want max Q for each state, where Q is just the difference 
        ## in the values of the possible state transition
        ## This is the policy evaluation phase
        for s in current_policy.keys(): # iterate over all states
            ## Compute Q for each possible state transistion
            ## Start by creating a list of the adjacent states.
            possible_s_prime = neighbors[s]
            neighbor_states = list(possible_s_prime.values())
            ## Check if terminal state is neighbor, but state is not terminal.
            if(terminal in neighbor_states and s != terminal):
                ## account for the special case adjacent to goal
                neighbor_Q = []
                for s_prime in possible_s_prime.keys(): # Iterate over adjacent states
                    if(neighbors[s][s_prime] == terminal):  
                         neighbor_Q.append(returns[s])
                    else: neighbor_Q.append(0.0) ## Other transisions have 0 value.   
            else: 
                 ## The other case is rather easy. Compute Q for the transistion to each neighbor           
                 neighbor_values = returns[neighbor_states]
                 neighbor_Q = [n_val - returns[s] for n_val in neighbor_values]
                
            ## Find the index for the state transistions with the largest values 
            ## May be more than one. 
            max_index = np.where(np.array(neighbor_Q) == max(neighbor_Q))[0]  
            
            ## Probabilities of transition
            ## Need to allow for further exploration so don't let any 
            ## transition probability be 0.
            ## Some gymnastics are required to ensure that the probabilities 
            ## over the transistions actual add to exactly 1.0
            neighbors_len = float(len(np.array(neighbor_Q)))
            max_len = float(len(max_index))
            diff = round(neighbors_len - max_len,3)
            prob_for_policy = round(1.0/max_len,3)
            adjust = round((epsilon * (diff)), 3)
            prob_for_policy = prob_for_policy - adjust
            if(diff != 0.0):
                remainder = (1.0 - max_len * prob_for_policy)/diff
            else:
                remainder = epsilon
                                                 
            for i, key in enumerate(current_policy[s]): ## Update policy
                if(i in max_index): current_policy[s][key] = prob_for_policy
                else: current_policy[s][key] = remainder          
                   
    return current_policy
 
 
nr.seed(9876)
MC_policy = MC_optimal_policy(start_state, policy, neighbors, rew_wall, terminal = 15, episodes = 50, cycles = 10, 
                              epsilon = 0.1)  
MC_policy

{0: {'d': 0.7,
  'l': 0.10000000000000002,
  'r': 0.10000000000000002,
  'u': 0.10000000000000002},
 1: {'d': 0.10000000000000002,
  'l': 0.10000000000000002,
  'r': 0.7,
  'u': 0.10000000000000002},
 2: {'d': 0.10000000000000002,
  'l': 0.10000000000000002,
  'r': 0.10000000000000002,
  'u': 0.7},
 3: {'d': 0.10000000000000002,
  'l': 0.7,
  'r': 0.10000000000000002,
  'u': 0.10000000000000002},
 4: {'d': 0.10000000000000002,
  'l': 0.10000000000000002,
  'r': 0.7,
  'u': 0.10000000000000002},
 5: {'d': 0.10000000000000002,
  'l': 0.7,
  'r': 0.10000000000000002,
  'u': 0.10000000000000002},
 6: {'d': 0.10000000000000002,
  'l': 0.10000000000000002,
  'r': 0.10000000000000002,
  'u': 0.7},
 7: {'d': 0.10000000000000002,
  'l': 0.10000000000000002,
  'r': 0.7,
  'u': 0.10000000000000002},
 8: {'d': 0.10000000000000002,
  'l': 0.7,
  'r': 0.10000000000000002,
  'u': 0.10000000000000002},
 9: {'d': 0.10000000000000002,
  'l': 0.10000000000000002,
  'r': 0.10000000000000002,
  'u': 0.7},


## 1.Create a simulator for the grid world environment. All interactions between your agents and the environment must be though calls to the function you create

In [166]:
def next_state(start_state, policy, neighbors, rewards, terminal):
    s = list(policy[start_state].keys())
    k = nr.choice(s, size = 1)[0]
    r = rew_wall[start_state][k]
    s_next = neighbors[start_state][k]
    done = False
    if s_next == terminal:
        done = True
    return (start_state,s_next,k,r,done)

In [167]:
done = False
start_state = 12
while done == False:
    (start_state,s_next,action,r,done) = next_state(start_state, policy, neighbors, rew_wall, 15)
    print("Current State = ",start_state,"Next State = ",s_next,"Action = ",action,"Reward = ",r,"Done = ",done)
    start_state = s_next

Current State =  12 Next State =  12 Action =  d Reward =  -10.0 Done =  False
Current State =  12 Next State =  12 Action =  l Reward =  -10.0 Done =  False
Current State =  12 Next State =  12 Action =  d Reward =  -10.0 Done =  False
Current State =  12 Next State =  8 Action =  u Reward =  -1.0 Done =  False
Current State =  8 Next State =  12 Action =  d Reward =  -1.0 Done =  False
Current State =  12 Next State =  13 Action =  r Reward =  -100.0 Done =  False
Current State =  13 Next State =  12 Action =  l Reward =  -1.0 Done =  False
Current State =  12 Next State =  12 Action =  d Reward =  -10.0 Done =  False
Current State =  12 Next State =  12 Action =  d Reward =  -10.0 Done =  False
Current State =  12 Next State =  8 Action =  u Reward =  -1.0 Done =  False
Current State =  8 Next State =  4 Action =  u Reward =  -1.0 Done =  False
Current State =  4 Next State =  4 Action =  l Reward =  -10.0 Done =  False
Current State =  4 Next State =  5 Action =  r Reward =  -1.0 D

### Markov - Control

In [412]:
import pandas as pd
from collections import OrderedDict
def mc_control(start_state, policy, neighbors, rewards, terminal, cycles, episodes,epsilon):
    cp = copy.deepcopy(policy)
    
    for j in range(0,cycles):
        nx = []
        act = []
        rew = []
        done = False
    
        for z in range(0,episodes): 
            done = False
            start_state = 12
            while done == False:
                (current_state,s_next,action,r,done) = next_state(start_state, policy, neighbors, rew_wall, 15)
                start_state = s_next
                nx.append(current_state)
                act.append(action)
                rew.append(r)
        df = pd.DataFrame(OrderedDict({'state':nx, 'action':act, 'reward':rew}))
        f1 = df.groupby(['state','action']).mean()
        f1.reset_index(inplace=True)
        max_df = f1.groupby(['state','action']).max()
        #f2 = max_df.reset_index()
        to_be_merged = max_df.groupby(['state']).max()
        to_be_merged.reset_index(inplace=True)
        f2 = f1.merge(to_be_merged, on=['state'])
        f3 = f2[f2.reward_x == f2.reward_y]
        #df = f3
        p = random.random()
        if p >0.5:
            df = f3.groupby('state').last().reset_index()
        else:
            df = f3.groupby('state').first().reset_index()
        #df = f3.groupby('state').sample(n=1).reset_index()
        #return(df)
        
        #Update Policy
        st = df.index.tolist()
        action = df.action.tolist()

        prob_for_policy = 1- epsilon
        remainder = epsilon
        for k in range(0,len(st)):
            for i, key in enumerate(cp[st[k]]): ## Update policy
                #print(i,key)
                if key == action[k]:
                    #cp[st[k]][key] = 1 - epsilon + (epsilon/np.absolute(r[k]))
                    cp[st[k]][key] = 0.7
                else:
                    #cp[st[k]][key] = epsilon
                    cp[st[k]][key] = 0.1
    
    
    return(cp)
        

In [414]:
df = mc_control(12, policy, neighbors, rewards, 15, 20,1000,0.1)
df

{0: {'d': 0.1, 'l': 0.1, 'r': 0.7, 'u': 0.1},
 1: {'d': 0.1, 'l': 0.1, 'r': 0.7, 'u': 0.1},
 2: {'d': 0.1, 'l': 0.1, 'r': 0.7, 'u': 0.1},
 3: {'d': 0.1, 'l': 0.7, 'r': 0.1, 'u': 0.1},
 4: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 5: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 6: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 7: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 8: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 9: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 10: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 11: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 12: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 13: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 14: {'d': 0.1, 'l': 0.1, 'r': 0.1, 'u': 0.7},
 15: {'d': 0.0, 'l': 0.0, 'r': 0.0, 'u': 0.0}}

## 2.Create and execute agents using the n-step TD method for value estimation and n-step SARSA (action value) for control